# Project 4:  Web Scraping for Retail Arbitrage

### Finding Underpriced RVs on Craigslist

Goal Statement: to programmatically compare the cost of one specific item available for sale on Craigslist in multiple different cities to calculate if a retail arbitrage business (buying in a cheaper location and reselling in a more expensive one) could hypothetically be lucrative. 

For the sake of clarity, I have examined if it is cheaper to buy elsewhere and sell in Los Angeles, California. 

In [1]:
# PREP REQUIRED LIBRARIES
import pandas as pd
import numpy as np
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from bs4 import BeautifulSoup
import requests



## 1)  Procure a list of the largest US cities from Wikipedia 

In [2]:
# SCRAPE WIKIPEDIA FOR LARGEST US CITIES (NON-EXHAUSTIVE LIST)
r = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population").text


In [3]:
import string

##get list of city names

aa = Selector(text=r).xpath('//*[@id="mw-content-text"]/table[4]//td[2]//text()').extract()
coords = Selector(text=r).xpath("//span[@class='geo']/text()").extract()
coords_pair = Selector(text=r).xpath("//span[@class='fn org']/text()").extract()


In [4]:
###clean the named outputs, make lowercase and no spaces for better string match

output = []
for i in coords_pair:      
    clean = ''
    for elm in i.split():                 
        if elm[0] in string.letters:
            clean+=elm.lower()
    output.append(clean)

    
output = [i.encode("ascii", "ignore")for i in output]
coords = [i.replace(";",",").encode("ascii") for i in coords]

city_coord_dict = dict(zip(output, coords))   


#### Clean The List

Filter out any cities with improper ASCII characters, narrow down to well formed cities. 

In [71]:
clean_names = []
for i in aa:
    try:
        i.encode("ascii")
        clean_names2.append(i)
    except UnicodeEncodeError: 
        pass



#### Capture current pricing information via Craigslist in one city.
Pull a city from the scraped data, get cooresponding city data from Craigslist, searching for "rv" in the auto section.  Pull out the prices.

In [32]:
url_test = "https://boulder.craigslist.org/search/rva?query=Jayco"

def get_city_avg(city, city_url):
    h = requests.get(city_url).text
#     bb = Selector(text=h).xpath('//*[@id="sortable-results"]//a/span//text()').extract()
    bb = Selector(text=h).xpath('//*[@id="sortable-results"]/ul//li/a/span/text()').extract()
    bb = [float(i.strip().replace("$",'')) for i in bb]
    city_avg = np.mean(np.asarray(bb))
    city_median = np.median(np.asarray(bb))
    return [city, city_avg, city_median]


get_city_avg("madison", url_test)

['madison', 20193.202127659573, 18995.0]


#### Create a function that creates search string URLs to repeat the procedure across cities.

Note: craigslist cities have distinct url constructions (ie: SFBay Area, NYC, Boston, Miami, Seattle, etc).  

Hence, the need for a mapping to connect wiki cities to craigslist values and compare. 

In [33]:
###Get the initial scrape, build out the url strings

jj = requests.get("https://www.craigslist.org/about/sites").text
urls= Selector(text=jj).xpath('//*[@id="pagecontainer"]/section/div[3]/div/ul//li/a/@href').extract()
cl_cities = Selector(text=jj).xpath('//*[@id="pagecontainer"]/section/div[3]/div/ul//li/a/text()').extract()
urls = [(i.encode("ascii").replace("//","https://") + "search/rva?query=Jayco")for i in urls]
cl_cities = [i.encode("ascii").replace(" ",'') for i in cl_cities]

master = zip(cl_cities, urls)

In [34]:
master

[('auburn', 'https://auburn.craigslist.org/search/rva?query=Jayco'),
 ('birmingham', 'https://bham.craigslist.org/search/rva?query=Jayco'),
 ('dothan', 'https://dothan.craigslist.org/search/rva?query=Jayco'),
 ('florence/muscleshoals',
  'https://shoals.craigslist.org/search/rva?query=Jayco'),
 ('gadsden-anniston', 'https://gadsden.craigslist.org/search/rva?query=Jayco'),
 ('huntsville/decatur',
  'https://huntsville.craigslist.org/search/rva?query=Jayco'),
 ('mobile', 'https://mobile.craigslist.org/search/rva?query=Jayco'),
 ('montgomery', 'https://montgomery.craigslist.org/search/rva?query=Jayco'),
 ('tuscaloosa', 'https://tuscaloosa.craigslist.org/search/rva?query=Jayco'),
 ('anchorage/mat-su',
  'https://anchorage.craigslist.org/search/rva?query=Jayco'),
 ('fairbanks', 'https://fairbanks.craigslist.org/search/rva?query=Jayco'),
 ('kenaipeninsula', 'https://kenai.craigslist.org/search/rva?query=Jayco'),
 ('southeastalaska', 'https://juneau.craigslist.org/search/rva?query=Jayco'),
 (

Make a dictionary with the city as key and the lat/lon and url as values

In [19]:
#if string for tuple[0] in dictionary of names and geos... get me the key 

cit_url_coords = {}
for tup in master:
    if tup[0] in city_coord_dict:
        cit_url_coords[tup[0]] = str(tup[1]), city_coord_dict[tup[0]]
cit_url_coords

{'abilene': ('https://abilene.craigslist.org/search/rva?query=Jayco',
  '32.4545, -99.7381'),
 'albuquerque': ('https://albuquerque.craigslist.org/search/rva?query=Jayco',
  '35.1056, -106.6474'),
 'amarillo': ('https://amarillo.craigslist.org/search/rva?query=Jayco',
  '35.1978, -101.8287'),
 'annarbor': ('https://annarbor.craigslist.org/search/rva?query=Jayco',
  '42.2756, -83.7313'),
 'athens': ('https://athensohio.craigslist.org/search/rva?query=Jayco',
  '33.9496, -83.3701'),
 'atlanta': ('https://atlanta.craigslist.org/search/rva?query=Jayco',
  '33.7629, -84.4227'),
 'augusta': ('https://augusta.craigslist.org/search/rva?query=Jayco',
  '33.3655, -82.0734'),
 'austin': ('https://austin.craigslist.org/search/rva?query=Jayco',
  '30.3072, -97.7560'),
 'bakersfield': ('https://bakersfield.craigslist.org/search/rva?query=Jayco',
  '35.3212, -119.0183'),
 'baltimore': ('https://baltimore.craigslist.org/search/rva?query=Jayco',
  '39.3002, -76.6105'),
 'batonrouge': ('https://batonrou

#### Run scrape and pickle out results 

In [40]:
def scrape_city_list(a_dict):
    avgs = [get_city_avg(i, a_dict[i][0]) for i in a_dict.keys()]
    return avgs

##this needs a dictionary, it will call "get city avg" for the key (location) and the location's 0 value in the dict
## which needs to be the url for get_city_avg to work

In [41]:
city_avgs_and_med = scrape_city_list(cit_url_coords)

[['houston', 20332.178571428572, 14000.0],
 ['madison', 12283.023255813954, 8500.0],
 ['tallahassee', 26263.444444444445, 18000.0],
 ['birmingham', 16205.192307692309, 12250.0],
 ['cincinnati', 14054.516129032258, 16495.0],
 ['annarbor', 17081.282051282051, 9900.0],
 ['newyorkcity', 16952.182795698925, 14500.0],
 ['fortwayne', 14347.985074626866, 11575.0],
 ['greenbay', 9840.0240963855413, 7500.0],
 ['clarksville', 20883.461538461539, 15500.0],
 ['denver', 20498.726315789474, 18495.0],
 ['lansing', 21769.471698113208, 17495.0],
 ['sanantonio', 21542.885416666668, 18347.5],
 ['eugene', 19886.755813953489, 16995.0],
 ['boulder', 20163.075268817203, 18995.0],
 ['tulsa', 23215.808510638297, 15400.0],
 ['amarillo', 26848.764705882353, 17472.5],
 ['jacksonville', 14148.276595744681, 12500.0],
 ['syracuse', 17958.434782608696, 12900.0],
 ['pueblo', 18184.958333333332, 17995.0],
 ['lincoln', 12268.421052631578, 9999.0],
 ['wilmington', 13740.183673469388, 12000.0],
 ['lexington', 26157.1066666

In [45]:
df_mean_med = pd.Series(city_avgs_and_med)
df_mean_med.to_csv('city_means_meds.csv',
              index=False)

In [46]:
## add mean and medians in to tuple dictionary values
for lst in df_mean_med:
    cit_url_coords[lst[0]] += (lst[1],) + (lst[2],)


In [47]:
##Convert to pandas
universe = pd.DataFrame.from_dict(cit_url_coords).T
universe.columns = ["url","lat_lon", "mean", "median"]



## 2) Regional comparisons

#### Calculate the price to transport using Google Distance API 


In [73]:
# read in API key

with open('/Users/valerieburchby/Desktop/Keys/p4_key.txt', 'r') as myfile:
    api_key=myfile.read().replace('\n','')



In [64]:
import googlemaps 
import json


def dist_to_la(a):
    gmaps = googlemaps.Client(api_key)
    distance_json = gmaps.distance_matrix(a,(34.0522, -118.2437), mode='driving', units = "imperial" )
    try:
        miles = distance_json["rows"][0]["elements"][0]["distance"]["text"]
        i = miles.split(" ")[0]
        i = int(i.replace(",","").encode("ascii"))
        return i
    except KeyError:
        return np.nan

    except ValueError:
        return np.nan
    



In [65]:
universe["to_la"] = (universe["lat_lon"]).apply(dist_to_la)

In [66]:
universe["trip_cost"] = (universe["to_la"]/10)*3.5
universe["oo_pocket"] = universe["median"] + universe["trip_cost"]


In [67]:
universe_sorted = universe.sort_values("oo_pocket",axis=0, na_position = "last")
universe_sorted.head()

,url,lat_lon,mean,median,to_la,trip_cost,oo_pocket
desmoines,https://desmoines.craigslist.org/search/rva?qu...,"41.5739, -93.6167",12325.6,6500,1684.0,589.40,7089.4
sandiego,https://sandiego.craigslist.org/search/rva?que...,"32.8153, -117.1350",18288.1,8200,115.0,40.25,8240.25
greenbay,https://greenbay.craigslist.org/search/rva?que...,"44.5207, -87.9842",9840.02,7500,2121.0,742.35,8242.35
charleston,https://charlestonwv.craigslist.org/search/rva...,"32.8179, -79.9589",13125.4,8000,2479.0,867.65,8867.65
madison,https://madison.craigslist.org/search/rva?quer...,"43.0878, -89.4301",12283,8500,1974.0,690.90,9190.9


In [68]:
###Inspect Los Angeles prices to compare 
universe_sorted.loc["losangeles"]

url          https://losangeles.craigslist.org/search/rva?q...
lat_lon                                     34.0194, -118.4108
mean                                                   24036.5
median                                                   13925
to_la                                                      NaN
trip_cost                                                  NaN
oo_pocket                                                  NaN
Name: losangeles, dtype: object

## Conclusions

Yeah, it absolutely makes sense to buy and drive compared to the median Jayco price in LA of 18,344. Some major east coast cities seem to have very low median prices for RVs, even when you account for the cost of driving it to Los Angeles. Factors not considered here: 

 * expense of my own personal time to drive it further distances
 * sense check for fraudlently low CL postings (like listing price as $1)
 * wear and tear of driving and life in a big east coast city might result in a crappier RV purchase